In [1]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import psycopg2

In [3]:
from dask.distributed import Client
from dask_kubernetes.operator import KubeCluster
cluster = KubeCluster(name='cluster2', image='ghcr.io/patmagauran/mldocker-base:main',env={'EXTRA_CONDA_PACKAGES':''}, resources={"requests": {"memory": "6Gi"}, "limits": {"memory": "6Gi"}}  )
#cluster = KubeCluster.from_name(name="foo")
#cluster.adapt(minimum=4, maximum=8)
cluster.scale(5)
# Scale up: connect to your own cluster with more resources
# see http://dask.pydata.org/en/latest/setup.html


In [4]:
client = Client(cluster)
client

<Client: 'tcp://10.56.1.4:8786' processes=5 threads=10, memory=30.00 GiB>

In [6]:
df = dd.read_parquet("gcs://ds352-data/rawData")
#df._meta.dtypes.to_dict()

In [7]:
#df.count().compute()

In [8]:
df = df.categorize(columns = [ 'County Code',
 'County Name',
 'Municipality Code',
 'Municipality Name',
 'District Code',
 'District Name',
 'Police Agency Code',
 'Police Agency Name',
 'Crash Year',
 'Illumination',
 'Weather',
 'Road Condition',
 'Collision Type',
 'Work Zone Type',
 'Work Zone Location',
 'Intersection Type',
 'Traffic Control Device',
 'Location Type',
 'Urban / Rural',

 'School Bus Indicator',
 'School Zone Indicator',

 'Lane Closed',
 'Lane Closure Direction',
 'Notify Highway Maintenance',
 'Special Jurisdiction',
 'Traffic Control Device Function',
 'Traffic Detour',
 'Workers Present',
 'Work Zone Close/Detour',
 'Work Zone Flagger',
 'Work Zone Law Officer',
 'Work Zone Closure',
 'Work Zone Moving',
 'Work Zone Other',
 'Work Zone Shoulder/Median',

 'Roadway Surface Type',
 'Interstate',
 'State Road',
 'Local Road Only',
 'Turnpike',
 'Wet Road',
 'Snow Slush Road',
 'Icy Road',
 'Sudden Deer',
 'Shoulder Related',
 'Rear End',
 'Head On / Opposite Direction Side Swipe',
 'Hit Fixed Object',
 'Single Vehicle Run Off Road',
 'Work Zone',
 'Property Damage Only',
 'Injury or Fatal',
 'Fatal or Suspected Serious Injury',
 'Injury',
 'Fatal',
 'Non Intersection',
 'Intersection',
 'Signalized Intersection',
 'Stop Controlled Intersection',
 'Unsignalized Intersection',
 'School Bus',
 'School Zone',
  'Hit Deer',
 'Hit Tree / Shrub',
 'Hit Embankment',
 'Hit Pole',
 'Hit Guide Rail',
 'Hit Guide Rail End',
 'Hit Barrier',
 'Hit Bridge',
 'Overturned',
 'Motorcycle',
 'Bicycle',
 'Heavy Truck Related',
 'Vehicle Failure',
 'Train / Trolley',
 'Phantom Vehicle',
 'Alcohol Related',
 'Drinking Driver',
 'Underage Drinking Driver',
 'Unlicensed',
 'Distracted',
 'Cell Phone',
 'No Clearance',
 'Running Red Light',
 'Tailgating',
 'Cross Median',
 'Curved Road',
 'Curve Driver Error',
 'Limit 65 MPH',
 'Speeding',
 'Speeding Related',
 'Aggressive Driving',
 'Fatigue / Asleep',
 'Driver 16 Years Old',
 'Driver 17 Years Old',
 'Driver 18 Years Old',
 'Driver 19 Years Old',
 'Driver 20 Years Old',
 'Driver 50 - 64 Years Old',
 'Driver 65 - 74 Years Old',
 'Driver 75 Plus',
 'Unbelted',
 'Pedestrian',
 'Commercial Vehicle',
 'PSP Reported',
 'NHTSA Aggressive Driving',
 'Deer Related',
 'Illumination Dark',
 'Running Stop Sign',
 'Train',
 'Trolley',
 'Hit Parked Vehicle',
 'Fire In Vehicle',
 'Vehicle Towed',
 'Hazardous Truck',
 'Suspected Serious Injury',
 'Suspected Minor Injury',
 'Possible Injury',
 'Motorcycle Drinking Driver',
 'Drug Related',
 'Illegal Drug Related',
 'School Bus Unit',
 'Drugged Driver',
 'Impaired Driver'])

In [9]:
#df._meta.dtypes.to_dict()

In [10]:
#mem2 = df.memory_usage(deep=True).compute()
#mem2.sum()

In [11]:
df = df.dropna(subset = ['Time of Day', 'Weather', 'Traffic Control Device', 'Intersection Type', 'Traffic Control Device Function', 'Illumination', 'Road Condition'])

In [12]:
df = df[dd.to_numeric(df['Time of Day'], errors='coerce').notnull()]

In [13]:
df = df.astype({'Time of Day': 'int32'})

In [14]:
df = df[df['Time of Day'] < 2400]

In [15]:
df = df.replace({"Yes": True, "No": False})
df = df.replace({"t": True, "f": False})
df['Lane Closed'] = df['Lane Closed'].replace({True: "Y",False: "N"})
df = df.fillna(value = {'Bicycle': False})
df['School Bus Indicator'] = df['School Bus Indicator'].replace({"Y": True, "N": False})
df['School Zone Indicator'] = df['School Zone Indicator'].replace({"Y": True, "N": False})

In [16]:
#df.count().compute()

In [17]:
df = df.astype({'School Bus Indicator': 'bool',
 'School Zone Indicator': 'bool',
  'Interstate': 'bool',
 'State Road': 'bool',
 'Local Road Only': 'bool',
 'Turnpike': 'bool',
 'Wet Road': 'bool',
 'Snow Slush Road': 'bool',
 'Icy Road': 'bool',
 'Sudden Deer': 'bool',
 'Shoulder Related': 'bool',
 'Rear End': 'bool',
 'Head On / Opposite Direction Side Swipe': 'bool',
 'Hit Fixed Object': 'bool',
 'Single Vehicle Run Off Road': 'bool',
 'Work Zone': 'bool',
 'Property Damage Only': 'bool',
 'Injury or Fatal': 'bool',
 'Fatal or Suspected Serious Injury': 'bool',
 'Injury': 'bool',
 'Fatal': 'bool',
 'Non Intersection': 'bool',
 'Intersection': 'bool',
 'Signalized Intersection': 'bool',
 'Stop Controlled Intersection': 'bool',
 'Unsignalized Intersection': 'bool',
 'School Bus': 'bool',
 'School Zone': 'bool',
 'Hit Deer': 'bool',
 'Hit Tree / Shrub': 'bool',
 'Hit Embankment': 'bool',
 'Hit Pole': 'bool',
 'Hit Guide Rail': 'bool',
 'Hit Guide Rail End': 'bool',
 'Hit Barrier': 'bool',
 'Hit Bridge': 'bool',
 'Overturned': 'bool',
 'Motorcycle': 'bool',
 'Bicycle': 'bool',
 'Heavy Truck Related': 'bool',
 'Vehicle Failure': 'bool',
 'Train / Trolley': 'bool',
 'Phantom Vehicle': 'bool',
 'Alcohol Related': 'bool',
 'Drinking Driver': 'bool',
 'Underage Drinking Driver': 'bool',
 'Unlicensed': 'bool',
 'Distracted': 'bool',
 'Cell Phone': 'bool',
 'No Clearance': 'bool',
 'Running Red Light': 'bool',
 'Tailgating': 'bool',
 'Cross Median': 'bool',
 'Curved Road': 'bool',
 'Curve Driver Error': 'bool',
 'Limit 65 MPH': 'bool',
 'Speeding': 'bool',
 'Speeding Related': 'bool',
 'Aggressive Driving': 'bool',
 'Fatigue / Asleep': 'bool',
 'Driver 16 Years Old': 'bool',
 'Driver 17 Years Old': 'bool',
 'Driver 18 Years Old': 'bool',
 'Driver 19 Years Old': 'bool',
 'Driver 20 Years Old': 'bool',
 'Driver 50 - 64 Years Old': 'bool',
 'Driver 65 - 74 Years Old': 'bool',
 'Driver 75 Plus': 'bool',
 'Unbelted': 'bool',
 'Pedestrian': 'bool',
 'Commercial Vehicle': 'bool',
 'PSP Reported': 'bool',
 'NHTSA Aggressive Driving': 'bool',
 'Deer Related': 'bool',
 'Illumination Dark': 'bool',
 'Running Stop Sign': 'bool',
 'Train': 'bool',
 'Trolley': 'bool',
 'Hit Parked Vehicle': 'bool',
 'Fire In Vehicle': 'bool',
 'Vehicle Towed': 'bool',
 'Hazardous Truck': 'bool',
 'Suspected Serious Injury': 'bool',
 'Suspected Minor Injury': 'bool',
 'Possible Injury': 'bool',
 'Motorcycle Drinking Driver': 'bool',
 'Drug Related': 'bool',
 'Illegal Drug Related': 'bool',
 'School Bus Unit': 'bool',
 'Drugged Driver': 'bool',
 'Impaired Driver': 'bool'})

In [18]:
#df._meta.dtypes.to_dict()

In [19]:
import pyarrow as pa
df.to_parquet("gs://ds352-data/processed-2",
             schema={
              "Municipality Code": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "Municipality Name": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "District Code": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "District Name": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "Police Agency Code": pa.dictionary(pa.uint16(), pa.string(), ordered=False),
                 "Police Agency Name": pa.dictionary(pa.uint16(), pa.string(), ordered=False)                 
             })

In [21]:
cluster.close()
client.close()